In [12]:
from pyspark import SparkContext
from operator import add
from math import log

In [6]:
sc = SparkContext()

In [7]:
rdd = sc.textFile('./project2_demo.txt')

## Word Count by Document based on doc id

In [8]:
frequency_by_document = rdd.flatMap(lambda line: [[word + "_" + line.split(' ')[0], 1] for word in line.split(' ')[1:] if word != ''])\
    .reduceByKey(lambda x, y: x + y)\
    .map(lambda x: [x[0].split("_")[1], [x[0].split("_")[0],  x[1]]])

## Term Frequency (First Hundred Entries)

In [9]:
term_frequency = rdd.map(lambda line: [line.split(' ')[0], len(line.split(' '))])\
                    .join(frequency_by_document)\
                    .map(lambda pair: [pair[1][1][0], [pair[0], pair[1][1][1]/pair[1][0]]])

## Interdocument Frequency

### Total Documents

In [10]:
total_documents = rdd.count()

### IDF

In [13]:
word_counts = rdd.map(lambda line: line.split(' ')[1:]).countByKey()
word_counts = sc.parallelize([[key, log(total_documents /value)] for key, value in word_counts.items()])

## Combined TF IDF (RDDs)

In [15]:
term_frequency.join(word_counts)\
    .map(lambda joined_pair: [joined_pair[0], joined_pair[1][1] , joined_pair[1][0][1], joined_pair[1][1] * joined_pair[1][0][1], 'document_id: '+ joined_pair[1][0][0]])\
    .collect()

[['erbb2',
  8.178825826840567,
  0.040983606557377046,
  0.3351977797885478,
  'document_id: 10022814'],
 ['erbb2',
  8.178825826840567,
  0.05,
  0.4089412913420284,
  'document_id: 10233167'],
 ['erbb2',
  8.178825826840567,
  0.06315789473684211,
  0.5165574206425622,
  'document_id: 10318765'],
 ['erbb2',
  8.178825826840567,
  0.0064516129032258064,
  0.05276661823768108,
  'document_id: 10908564'],
 ['erbb2',
  8.178825826840567,
  0.06842105263157895,
  0.5596038723627756,
  'document_id: 11062025'],
 ['erbb2',
  8.178825826840567,
  0.011560693641618497,
  0.09455289973226089,
  'document_id: 11156083'],
 ['erbb2',
  8.178825826840567,
  0.0029850746268656717,
  0.024414405453255426,
  'document_id: 11881999'],
 ['erbb2',
  8.178825826840567,
  0.009708737864077669,
  0.07940607598874337,
  'document_id: 12061997'],
 ['erbb2',
  8.178825826840567,
  0.006993006993006993,
  0.05719458620168229,
  'document_id: 12610629'],
 ['erbb2',
  8.178825826840567,
  0.011494252873563218,
